In [10]:
from googlesearch import search
import pandas as pd
import os
import requests
import urllib.parse as parse
from bs4 import BeautifulSoup
from tika import parser
from interruptingcow import timeout


data_path = '/Users/kylieleblancKylie/domino/dofis/data/schedules'

In [2]:
def remove_whitespace(df, text_col):
    '''
    Remove all \n, \t, \r characters from raw text
    '''
    col = text_col
    clean_text = df[col].map(lambda x: ' '.join(x.split()))
    df = df.copy()
    df[col] = clean_text
    df[col] = df[col].str.replace('â\x80\x99s', ' ')
    df[col] = df[col].str.replace('Ã¢ \x80 ¢', ' ')
    df[col] = df[col].str.replace('\x80', ' ')
    df[col] = df[col].str.replace('\x99s', ' ')
    df[col] = df[col].str.replace('Ã\x82', ' ')
    df[col] = df[col].str.replace('\u200b', ' ')
    df[col] = df[col].str.replace('Â', ' ')
    df[col] = df[col].str.replace('¤', ' ')
    df[col] = df[col].str.replace('â', ' ')
    df[col] = df[col].str.replace('', ' ')
    df[col] = df[col].str.replace('¢', ' ')
    df[col] = df[col].str.replace('ï·', ' ')
    df[col] = df[col].str.replace('â', ' ')
    df[col] = df[col].str.replace('Ã', ' ')
    df[col] = df[col].str.replace('Â', ' ')
    df[col] = df[col].str.replace('¨', ' ')
    df[col] = df[col].str.replace('¨', ' ')


    return df

## Gather results from Google search

In [3]:
schools = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'tea', 'desc_c_yr1819.csv'))
schools['full'] = schools["campname"].str.replace(' JR H S', "junior high school") 
schools['full'] = schools["full"].str.replace(' H S', " high school") 
schools['full'] = schools["full"].str.replace(' EL', " elementary school") 
schools['full'] = schools["full"].str.replace(' MIDDLE', " middle school") 
schools['full'] = schools["full"].str.replace(' MIDDLE', " middle school") 
schools['full'] = schools["full"].str.replace(' INT', " intermediate school") 
schools['full'] = schools["full"].str.replace(' J H', " junior high school") 
schools['full'] = schools["full"].str.replace(' PRI', " primary school") 
schools['full'] = schools["full"].str.replace(' CTR', " center") 
schools['full'] = schools["full"].str.replace(' SCHOOL', " school") 
schools['full'] = schools["full"].str.replace(' SCH', " school") 


schools = schools[['campname', 'full']].sample(10, random_state = 3452)
#schools = schools[['campname', 'full']]

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
link1 = []
link2 = []
link3 = []
for campus in schools.full:
    query = campus + " bell schedule"
    print(query)
    urls = []
    for j in search(query, tld="co.in", num=10, stop=3, pause=2): 
        urls.append(j)
        print(j)
    link1.append(urls[0])
    link2.append(urls[1])
    link3.append(urls[2])
link1

SHADYDALE elementary school bell schedule
https://www.houstonisd.org/shadydale
https://www.publicschoolreview.com/shadydale-elementary-school-profile
https://www.har.com/school/101912479/shadydale-elementary-school
SNOW HEIGHTS elementary school bell schedule
https://www.birdvilleschools.net/SnowHeightsElementary
https://www.birdvilleschools.net/Page/11109
https://www.birdvilleschools.net/times
GLEN ROSE junior high school bell schedule
http://www.grisd.net/campuses/glen-rose-junior-high/grjh-students/grjh-bell-schedules/
http://www.grisd.net/campuses/glen-rose-junior-high/
http://www.grisd.net/
JUAN SEGUIN elementary school bell schedule
https://seguin-es.weatherfordisd.com/apps/bell_schedules/
https://seguin-es.weatherfordisd.com/apps/bell_schedules/printerfriendly.jsp
https://www.aisd.net/seguin-high-school/about/bell-schedule/
HARMONY junior high school bell schedule
https://www.lcps.org/domain/13059
https://www.lcps.org/cms/lib/VA01000195/Centricity/Domain/2019/Harmony%20Bell%20Sc

['https://www.houstonisd.org/shadydale',
 'https://www.birdvilleschools.net/SnowHeightsElementary',
 'http://www.grisd.net/campuses/glen-rose-junior-high/grjh-students/grjh-bell-schedules/',
 'https://seguin-es.weatherfordisd.com/apps/bell_schedules/',
 'https://www.lcps.org/domain/13059',
 'https://www.industrialisd.org/Domain/10',
 'https://lamaramaisd.sharpschool.com/resources/for_parents/bell_schedules',
 'https://www.pearsallhs.org/apps/bell_schedules/printerfriendly.jsp',
 'https://rms.sbcisd.net/apps/bell_schedules/printerfriendly.jsp',
 'http://www.leakeyisd.net/District/1230-Bell-Schedule.html']

In [5]:
schools['link1'] = link1
schools['link2'] = link2
schools['link3'] = link3
schools

,campname,full,link1,link2,link3
4494,SHADYDALE EL,SHADYDALE elementary school,https://www.houstonisd.org/shadydale,https://www.publicschoolreview.com/shadydale-e...,https://www.har.com/school/101912479/shadydale...
7327,SNOW HEIGHTS EL,SNOW HEIGHTS elementary school,https://www.birdvilleschools.net/SnowHeightsEl...,https://www.birdvilleschools.net/Page/11109,https://www.birdvilleschools.net/times
7162,GLEN ROSE J H,GLEN ROSE junior high school,http://www.grisd.net/campuses/glen-rose-junior...,http://www.grisd.net/campuses/glen-rose-junior...,http://www.grisd.net/
5289,JUAN SEGUIN EL,JUAN SEGUIN elementary school,https://seguin-es.weatherfordisd.com/apps/bell...,https://seguin-es.weatherfordisd.com/apps/bell...,https://www.aisd.net/seguin-high-school/about/...
8183,HARMONY J H,HARMONY junior high school,https://www.lcps.org/domain/13059,https://www.lcps.org/cms/lib/VA01000195/Centri...,https://hrhs.osceolaschools.net/UserFiles/Serv...
5508,INDUSTRIAL EL EAST,INDUSTRIAL elementary school EAST,https://www.industrialisd.org/Domain/10,https://www.industrialisd.org/Page/1543,https://greenbriareastes.fcps.edu/about/bell-s...
499,LAMAR EL,LAMAR elementary school,https://lamaramaisd.sharpschool.com/resources/...,http://lamar.conroeisd.net/,http://lamar.conroeisd.net/department/faculty-...
3425,PEARSALL INT,PEARSALL intermediate school,https://www.pearsallhs.org/apps/bell_schedules...,https://www.pearsallhs.org/apps/bell_schedules/,https://pearsalljh.org/apps/bell_schedules/pri...
3057,RIVERSIDE MIDDLE,RIVERSIDE middle school,https://rms.sbcisd.net/apps/bell_schedules/pri...,https://www.yisd.net/domain/3836,https://www.greenville.k12.sc.us/rms/
6907,LEAKEY SCHOOL,LEAKEY school,http://www.leakeyisd.net/District/1230-Bell-Sc...,http://www.leakeyisd.org/?a=events&s=district&...,http://www.leakeyisd.org/District/Department/1...


# Extract Text

In [6]:
texts = []
for link in link1:
    with timeout(15, exception=RuntimeError):
        try:
            resp = requests.get(link)
            parsed = parser.from_buffer(resp.content)
            text = parsed['content']
            texts.append(text)
        except:
            texts.append('timeout')
texts
schools['text1'] = texts
schools = remove_whitespace(schools, 'text1')
schools.text1.head()

4494    Shadydale Elementary School / Ram Review Skip ...
7327    Snow Heights Elementary / Overview Skip to Mai...
7162    GRJH Bell Schedules – Glen Rose ISD Skip to co...
5289    Juan Seguin Elementary School Juan Seguin Elem...
8183    Bell Schedules / Bell Schedules Skip to Main C...
Name: text1, dtype: object

In [7]:
texts = []
for link in link2:
    with timeout(15, exception=RuntimeError):
        try:
            resp = requests.get(link)
            parsed = parser.from_buffer(resp.content)
            text = parsed['content']
            texts.append(text)
        except:
            texts.append('timeout')
texts
schools['text2'] = texts
schools = remove_whitespace(schools, 'text2')
schools.text2.head()

4494    Shadydale Elementary School Profile (2019-20) ...
7327    Snow Heights Elementary / Calendar Skip to Mai...
7162    Glen Rose Junior High – Glen Rose ISD Skip to ...
5289    Juan Seguin Elementary School — Bell Schedules...
8183    8:15 AM - Main Entrance unlocked for Students ...
Name: text2, dtype: object

In [8]:
texts = []
for link in link3:
    with timeout(15, exception=RuntimeError):
        try:
            resp = requests.get(link)
            parsed = parser.from_buffer(resp.content)
            text = parsed['content']
            texts.append(text)
        except:
            texts.append('timeout')
texts
schools['text3'] = texts
schools = remove_whitespace(schools, 'text3')
schools.text3.head()

4494    Shadydale Elementary School Houston, TX - HAR....
7327    About Birdville ISD / School Times Skip to Mai...
7162    Glen Rose ISD – Home of the Tigers Skip to con...
5289    Sucuri WebSite Firewall - Access Denied Websit...
8183    1st Bell 7:00 1st Period 7:05 - 7:54 *1 min. e...
Name: text3, dtype: object

## Save

In [11]:
schools.to_csv(os.path.join(data_path, 'schedule_link_text.csv'))

In [12]:
links = pd.read_csv(os.path.join(data_path, 'schedule_link_text.csv'))
links.head()

,Unnamed: 0,campname,full,link1,link2,link3,text1,text2,text3
0,4494,SHADYDALE EL,SHADYDALE elementary school,https://www.houstonisd.org/shadydale,https://www.publicschoolreview.com/shadydale-e...,https://www.har.com/school/101912479/shadydale...,Shadydale Elementary School / Ram Review Skip ...,Shadydale Elementary School Profile (2019-20) ...,"Shadydale Elementary School Houston, TX - HAR...."
1,7327,SNOW HEIGHTS EL,SNOW HEIGHTS elementary school,https://www.birdvilleschools.net/SnowHeightsEl...,https://www.birdvilleschools.net/Page/11109,https://www.birdvilleschools.net/times,Snow Heights Elementary / Overview Skip to Mai...,Snow Heights Elementary / Calendar Skip to Mai...,About Birdville ISD / School Times Skip to Mai...
2,7162,GLEN ROSE J H,GLEN ROSE junior high school,http://www.grisd.net/campuses/glen-rose-junior...,http://www.grisd.net/campuses/glen-rose-junior...,http://www.grisd.net/,GRJH Bell Schedules – Glen Rose ISD Skip to co...,Glen Rose Junior High – Glen Rose ISD Skip to ...,Glen Rose ISD – Home of the Tigers Skip to con...
3,5289,JUAN SEGUIN EL,JUAN SEGUIN elementary school,https://seguin-es.weatherfordisd.com/apps/bell...,https://seguin-es.weatherfordisd.com/apps/bell...,https://www.aisd.net/seguin-high-school/about/...,Juan Seguin Elementary School Juan Seguin Elem...,Juan Seguin Elementary School — Bell Schedules...,Sucuri WebSite Firewall - Access Denied Websit...
4,8183,HARMONY J H,HARMONY junior high school,https://www.lcps.org/domain/13059,https://www.lcps.org/cms/lib/VA01000195/Centri...,https://hrhs.osceolaschools.net/UserFiles/Serv...,Bell Schedules / Bell Schedules Skip to Main C...,8:15 AM - Main Entrance unlocked for Students ...,1st Bell 7:00 1st Period 7:05 - 7:54 *1 min. e...


In [14]:
links.campname.nunique()

10

In [17]:
for link in links[(links.campname == "LAMAR EL")].link1:
    print(link)

https://lamaramaisd.sharpschool.com/resources/for_parents/bell_schedules


In [18]:
len(links)

10